# JAP

In [26]:
#this is working!!!
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output

# Load the dataframes
sources_df = pd.read_csv("sources25.csv", encoding='latin-1')
sjr_df = pd.read_csv("SJR23.csv", encoding='latin-1')
matrix_sco_df = pd.read_csv("matrixSCO.csv", encoding='latin-1')

# Extract unique journal titles and ISSNs
title_options = sources_df['Title'].dropna().unique().tolist()
issn_options = sources_df['ISSN'].dropna().astype(str).apply(lambda x: x.lstrip('0')).tolist()
eissn_options = sources_df['EISSN'].dropna().astype(str).apply(lambda x: x.lstrip('0')).tolist()
all_options = title_options + issn_options + eissn_options

# Create the autocomplete widget
search_widget = widgets.Combobox(
    placeholder='Type journal name or ISSN (skip 0s)',
    options=all_options,
    ensure_option=True,
    description='Search:',
    layout=widgets.Layout(width='400px')  # Increased width
)

display(search_widget)

def display_journal_info(change):
    clear_output(wait=True)
    query = change['new'].strip().lstrip('0')  # Remove all leading zeros

    # Check for matches, ignoring leading zeros
    sources_df['ISSN_clean'] = sources_df['ISSN'].astype(str).apply(lambda x: x.lstrip('0'))
    sources_df['EISSN_clean'] = sources_df['EISSN'].astype(str).apply(lambda x: x.lstrip('0'))

    if query in sources_df['Title'].values:
        source_info = sources_df[sources_df['Title'] == query].iloc[0]
    elif query in sources_df['ISSN_clean'].values:
        source_info = sources_df[sources_df['ISSN_clean'] == query].iloc[0]
    elif query in sources_df['EISSN_clean'].values:
        source_info = sources_df[sources_df['EISSN_clean'] == query].iloc[0]
    else:
        print("No matching journal found.")
        display(search_widget)
        return

    title, issn, eissn, sid = source_info[['Title', 'ISSN', 'EISSN', 'SID']]

    # Get info from sjr_df
    sjr_info = sjr_df[sjr_df['SID'] == sid]

    # Display basic journal information
    print(f"Title: {title}")
    print(f"ISSN: {issn}")
    print(f"EISSN: {eissn}")

    if not sjr_info.empty:
        # Print SJR only once for the first row
        first_row = sjr_info.iloc[0]  # Get the first row
        print(f"SJR(2023): {first_row['SJR']}")

        for _, row in sjr_info.iterrows():
            print(f"Subject Area: {row['Description']}")
            print(f"ASJC: {row['ASJC']}")
            print(f"Quartile: {row['Quartile']}")

            # Get ASJC code and extract first two digits
            asjc_code = str(row['ASJC'])[:2]

            # Filter matrix_sco_df based on ASJC code
            matching_rows = matrix_sco_df[matrix_sco_df['ASJC'].astype(str).str.startswith(asjc_code)]

            if not matching_rows.empty:
                # Get unique PN values for the matching rows
                pn_values = matching_rows['PN'].dropna().unique().tolist()
                print(f"Професионално направление: {', '.join(map(str, pn_values))}")
            else:
                print(f"No matching PN values found for {title} (ASJC: {asjc_code})")

    display(search_widget)  # Re-display the widget

# Observe the widget for changes
search_widget.observe(display_journal_info, names='value')


Combobox(value='', description='Search:', ensure_option=True, layout=Layout(width='400px'), options=('@GRH', '…